<a href="https://colab.research.google.com/github/mahbodnr/FastSpike/blob/main/MLP_MIN2Netipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[<img src="https://min2net.github.io/assets/images/min2net-logo.png" width="30%" height="30%">](https://min2net.github.io)

### End-to-End Multi-Task Learning for Subject-Independent Motor Imagery EEG Classification

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1IE5J0Yn10ZIhWjSatQn_QWJWZblr6tZy?usp=sharing)
[![Pypi Downloads](https://img.shields.io/pypi/v/min2net?color=green&logo=pypi&logoColor=white)](https://pypi.org/project/min2net)
[![DOI](https://img.shields.io/badge/DOI-10.1109%2FTBME.2021.3137184-blue)](https://ieeexplore.ieee.org/document/9658165)

Python API and the novel algorithm for motor imagery EEG recognition named MIN2Net. The API benefits BCI researchers ranging from beginners to experts. We demonstrate the examples in using the API for loading benchmark datasets, preprocessing, training, and validation of SOTA models, including MIN2Net. In summary, the API allows the researchers to construct the pipeline for benchmarking the newly proposed models and very recently developed SOTA models.


- **Website:** [https://min2net.github.io](https://min2net.github.io)
- **Documentation:** [https://min2net.github.io](https://min2net.github.io)
- **Source code:** [https://github.com/IoBT-VISTEC/MIN2Net](https://github.com/IoBT-VISTEC/MIN2Net)
- **Bug reports:** [https://github.com/IoBT-VISTEC/MIN2Net/issues](https://github.com/IoBT-VISTEC/MIN2Net/issues)

## Installation

In [ ]:
! pip install min2net

## Load and preprocess dada

In [ ]:
import min2net
import min2net.preprocessing as prep


min2net.utils.load_raw('BCIC2a')
prep.BCIC2a.time_domain.subject_dependent_setting(k_folds=5,
                                                 pick_smp_freq=100, 
                                                 bands=[8, 30], 
                                                 order=5, 
                                                 save_path='datasets')

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.1.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


## Creare DataLoader

In [ ]:
from min2net.utils import DataLoader

loader = DataLoader(dataset='BCIC2a', 
                    train_type='subject_dependent', 
                    subject=1, 
                    data_format='NTCD', # for MIN2Net
                    data_type='time_domain', 
                    dataset_path='datasets')

## MIN2Net model example

In [ ]:
# load dataset
X_train, y_train = loader.load_train_set(fold=1)
X_val, y_val = loader.load_val_set(fold=1)
X_test, y_test = loader.load_test_set(fold=1)


from min2net.model import MIN2Net
model = MIN2Net(input_shape=(1, 400, 20), num_class=2, monitor='val_loss', shuffle=True)
model.fit(X_train, y_train, X_val, y_val)
Y, evaluation = model.predict(X_test, y_test)

change data_format to 'NTCD', new dimention is (115, 1, 400, 20)
change data_format to 'NTCD', new dimention is (29, 1, 400, 20)
change data_format to 'NTCD', new dimention is (144, 1, 400, 20)
Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 400, 20)]      0         
                                                                 
 conv2d (Conv2D)             (None, 1, 400, 20)        25620     
                                                                 
 batch_normalization (BatchN  (None, 1, 400, 20)       80        
 ormalization)                                                   
                                                                 
 average_pooling2d (AverageP  (None, 1, 100, 20)       0         
 ooling2D)                                                       
                                                               

## EEGNet model example
https://iopscience.iop.org/article/10.1088/1741-2552/aace8c

In [ ]:
loader = DataLoader(dataset='BCIC2a', 
                    train_type='subject_dependent', 
                    subject=1, 
                    data_format='NCTD', # for EEGNet and DeepConvNet (our paper and the original paper set data_format='NDCT')
                    data_type='time_domain', 
                    dataset_path='datasets')

In [ ]:
# load dataset
X_train, y_train = loader.load_train_set(fold=1)
X_val, y_val = loader.load_val_set(fold=1)
X_test, y_test = loader.load_test_set(fold=1)

from min2net.model import EEGNet
# (our paper and the original paper set input_shape=(1,20,400), data_format='channels_first')
model = EEGNet(input_shape=(20,400,1), num_class=2, dropout_rate=0.25, shuffle=True, data_format='channels_last')
model.fit(X_train, y_train, X_val, y_val)
Y, evaluation = model.predict(X_test, y_test)

change data_format to 'NCTD', new dimention is (115, 20, 400, 1)
change data_format to 'NCTD', new dimention is (29, 20, 400, 1)
change data_format to 'NCTD', new dimention is (144, 20, 400, 1)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20, 400, 1)]      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 20, 400, 8)        1600      
                                                                 
 batch_normalization_4 (Batc  (None, 20, 400, 8)       32        
 hNormalization)                                                 
                                                                 
 depthwise_conv2d (Depthwise  (None, 1, 400, 16)       320       
 Conv2D)                                                         
                                                                 

## DeepConvNet model example
https://doi.org/10.1002/hbm.2373

In [ ]:
# load dataset
X_train, y_train = loader.load_train_set(fold=1)
X_val, y_val = loader.load_val_set(fold=1)
X_test, y_test = loader.load_test_set(fold=1)

from min2net.model import DeepConvNet
# (our paper and the original paper set input_shape=(1,20,400), data_format='channels_first')
model = DeepConvNet(input_shape=(20,400,1), num_class=2, dropout_rate=0.25, shuffle=True, data_format='channels_last')
model.fit(X_train, y_train, X_val, y_val)
Y, evaluation = model.predict(X_test, y_test)

change data_format to 'NCTD', new dimention is (115, 20, 400, 1)
change data_format to 'NCTD', new dimention is (29, 20, 400, 1)
change data_format to 'NCTD', new dimention is (144, 20, 400, 1)
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 20, 400, 1)]      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 20, 396, 25)       150       
                                                                 
 conv2d_7 (Conv2D)           (None, 1, 396, 25)        12525     
                                                                 
 batch_normalization_10 (Bat  (None, 1, 396, 25)       4         
 chNormalization)                                                
                                                                 
 activation_4 (Activation)   (None, 1, 396, 25)        0       

## SpectralSpatialCNN model example
https://doi.org/10.1109/TNNLS.2019.2946869

In [ ]:
# this model requires spectral-spatial-mapping
# see https://min2net.github.io/docs/preprocessing/BCIC2a/#spectral-spatial-mapping

# generate fake data
import numpy as np
X_train = np.random.rand(100,20,28,28,1)
y_train = np.concatenate(([0]*50, [1]*50))
X_val = np.random.rand(40,20,28,28,1)
y_val = np.concatenate(([0]*20, [1]*20))
X_test = np.random.rand(40,20,28,28,1)
y_test = np.concatenate(([0]*20, [1]*20))

from min2net.model import SpectralSpatialCNN
model = SpectralSpatialCNN(input_shape=(28, 28, 1), num_class=2, epochs=10, dropout_rate=0.25, shuffle=True)
model.fit(X_train, y_train, X_val, y_val)
Y, evaluation = model.predict(X_test, y_test)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                            